In [1]:
import AdventOfCode
import qualified Data.Map.Strict as M

input <- readInt <$> dayString 3

In [2]:
import Data.Function.Memoize

genDRDiagonal = memoize $ \n -> (^2) .head $ drop n [1,3..]
genURDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - ((s-1)*3)
genULDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - ((s-1)*2)
genDLDiagonal n = let
    drDiagonal = genDRDiagonal n
    s = floor . sqrt $ fromIntegral drDiagonal
    in drDiagonal - (s-1)

In [3]:
getPos n = let
    diagonals = takeWhile (<n) $ map (^2) [1,3..]
    pos = length diagonals
    dr = genDRDiagonal pos
    dl = genDLDiagonal pos
    ur = genURDiagonal pos
    ul = genULDiagonal pos
    in case () of
    _ | n == dr           -> (pos, -pos)
      | n < dr && n >= dl -> (-pos + (n - dl), -pos)
      | n < dl && n >= ul -> (-pos, pos - (n - ul))
      | n < ul && n >= ur -> (pos - (n - ur), pos)
      | n < ur            -> (pos, pos - (ur - n))
    

-- Solve for distance.
(x,y) = getPos input
abs x + abs y

438

In [4]:
neighbours (x, y) =
    [ (x-1, y-1), (x, y-1), (x+1, y-1)
    , (x-1, y),             (x+1, y)
    , (x-1, y+1), (x, y+1), (x+1, y+1)
    ]

sumNeighbours g c = sum $ map (\c' -> M.findWithDefault 0 c' g) $ neighbours c

grid = M.fromList [((0,0), 1)]

loop gr i stop = let
    coord = getPos i
    value = sumNeighbours gr coord
    in if value > stop
        then (value, gr)
        else loop (M.insert coord value gr) (i+1) stop
        
fst $ loop grid 2 input

266330